### Necessary Imports

In [152]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests

### Use Beautiful Soup to pull all contents of the web-page

In [153]:
page = requests.get('https://en.wikipedia.org/wiki/UEFA_Cup_and_Europa_League_records_and_statistics')
soup = BeautifulSoup(page.content,'html.parser')

### Find the necessary table on the page. First let us consider the finalist data.

In [154]:
tables = soup.findAll("table")

### Build Table Header

In [155]:
table_header = tables[0].findAll("tr")
header_list=[]
for i in range(1,len(table_header[0])):
    if(i==1):
        header_list.append(table_header[0].contents[i].contents[0].lstrip().rstrip())
    elif (i%2 != 0):
        header_list.append(table_header[0].contents[i].contents[0].lstrip().rstrip())

In [156]:
header_list.append('League')

In [157]:
header_list

['Club', 'Winners', 'Runners-up', 'Years won', 'Years runner-up', 'League']

### Create an empty data frame with all the columns

In [158]:
el_final_record = pd.DataFrame(columns=header_list)

### Function to pull League information

In [159]:
def get_league(temp_link):
    club_link = 'https://en.wikipedia.org/' + temp_link
    club_page = requests.get(club_link)
    club_soup = BeautifulSoup(club_page.content,'html.parser')
    info_table = club_soup.find('table',attrs={"class":"infobox vcard"})
    info_table_rows = info_table.find_all('tr')
    try:
        for j in range(0,12):
            if(info_table_rows[j].contents[0].contents[0]=='League'):
                league_name=(info_table_rows[j].contents[1].contents[0].get('title'))
                break
    except IndexError:
        league_name = 'Unavailable'
    return(league_name)

### Consume all the row element columns and populate the data frame for the finalists

In [160]:
rows = tables[0].findAll("tr")
for club in range(1,len(rows)):
    club_name = rows[club].contents[1].contents[2].contents[0]
    temp_link = rows[club].contents[1].contents[2].get('href')
    league_name = get_league(temp_link)    
    titles = rows[club].contents[3].contents[0].rstrip()
    runners = rows[club].contents[5].contents[0].rstrip()
    x = rows[club].contents[7].findAll("a")
    win_list = []
    for i in range(0,len(x)):
        win_list.append(x[i].contents[0])
    y = rows[club].contents[9].findAll("a")
    runner_list = []
    for j in range(0,len(y)):
        runner_list.append(y[j].contents[0])
    el_final_record.loc[club-1] = [club_name,titles,runners,win_list,runner_list,league_name]

### Find the data for the semi-finalists and build a seperate data frame

In [161]:
semi_table_hdr = tables[7].findAll("tr")
semi_hdr_list = []
for i in range(1,len(semi_table_hdr[0])):
    if (i%2 != 0):
        semi_hdr_list.append(semi_table_hdr[0].contents[i].contents[0].lstrip().rstrip())

In [162]:
semi_hdr_list.append('League')

### Consume all the row element columns and populate the data frame for the finalists

In [163]:
el_semi_record = pd.DataFrame(columns=semi_hdr_list)
rows = tables[7].findAll("tr")
for club in range(1,len(rows)):
    club_name = rows[club].contents[1].contents[2].contents[0]
    temp_link = rows[club].contents[1].contents[2].get('href')
    league_name = get_league(temp_link)
    number_of_semis = rows[club].contents[3].contents[0].rstrip()
    x = rows[club].contents[5].findAll("a")
    years = []
    for i in range(0,len(x)):
        years.append(x[i].contents[0])
    el_semi_record.loc[club-1] = [club_name,number_of_semis,years,league_name]

In [164]:
season_dict={}
league_dict={}
for i in range(0,len(el_final_record)):
    club_name = el_final_record['Club'][i]
    league_name = el_final_record['League'][i]
    winning_list = el_final_record[el_final_record.Club == club_name]['Years won'].tolist()
    for i in range(0,len(winning_list)):
            season_list = winning_list[i]
            if(len(season_list)!=0):
                for i in season_list:
                    season_dict[i] = [club_name]
                    league_dict[i] = [league_name]

### Using the other website to get the runner-up details

In [165]:
supp_page = requests.get('https://en.wikipedia.org/wiki/List_of_UEFA_Cup_and_Europa_League_finals')
supp_soup = BeautifulSoup(supp_page.content,'html.parser')
supp_tables = supp_soup.findAll("table")
x=supp_tables[2].find_all("th")
hdr_list = ['Season','Runners-Up']
runners = pd.DataFrame(columns=hdr_list)
rn_league = pd.DataFrame(columns=hdr_list)
season_list=[]
for i in range(8,56):
    season = (x[i].contents[0].contents[0])
    if len(season) > 7:
        season_list.append(season[-4:])
    else:
        season_list.append(season[0:2]+season[-2:])
con = supp_tables[2].find_all("td")
runners.Season = season_list
rn_league.Season = season_list
runners_list=[]
for i in range(3,264,10):
    runners_list.append(con[i].contents[0])
for i in range(270,414,7):
    runners_list.append(con[i].contents[0])
runner_up=[]
league_run=[]
for i in range(0,len(runners_list)):
    team_name = runners_list[i].contents[0]
    runner_up.append(team_name)    
    temp_link = runners_list[i].get('href')
    league_name = get_league(temp_link)
    league_run.append(league_name)
runners['Runners-Up'] = runner_up
rn_league['Runners-Up'] = league_run
for i in range(0,len(runners)):
    season_dict[runners['Season'][i]].append(runners['Runners-Up'][i])
    league_dict[rn_league['Season'][i]].append(rn_league['Runners-Up'][i])

### Now add the other semi-finalists from the semi finalist data frame to complete the dataset

In [166]:
teams = el_semi_record['Team']
for team in teams:
    league = el_semi_record[el_semi_record['Team']==team]['League'].values[0]
    season_list=[]
    team_list = el_semi_record[el_semi_record['Team']==team]['Years']
    team_index = team_list.index[0]
    season_list = team_list[team_index]
    for season in season_list:
        if(team!=season_dict[season][0]):
            if(team!=season_dict[season][1]):
                season_dict[season].append(team)
                league_dict[season].append(league)

### Clean the data-set to remove duplication caused due to the use of different names for the same team

In [167]:
season_dict_items = season_dict.items()
for i in season_dict_items:
    if (len(i[1])>4):
        if((season_dict[i[0]][2][0:5]==season_dict[i[0]][0][0:5]) or (season_dict[i[0]][2][0:5]==season_dict[i[0]][1][0:5])):
            del(season_dict[i[0]][2])
            del(league_dict[i[0]][2])
        elif((season_dict[i[0]][3][0:5]==season_dict[i[0]][0][0:5]) or (season_dict[i[0]][3][0:5]==season_dict[i[0]][1][0:5])):
            del(season_dict[i[0]][3])
            del(league_dict[i[0]][3])
        elif((season_dict[i[0]][4][0:5]==season_dict[i[0]][0][0:5]) or (season_dict[i[0]][4][0:5]==season_dict[i[0]][1][0:5])):
            del(season_dict[i[0]][4])
            del(league_dict[i[0]][4])

### Clean the data-set to remove duplication caused due to the use of different names for the same team

In [168]:
for i in season_dict_items:
    if (len(i[1])>4):
        if((season_dict[i[0]][2][-6:]==season_dict[i[0]][0][-6:]) or (season_dict[i[0]][2][-6:]==season_dict[i[0]][1][-6:])):
            del(season_dict[i[0]][2])
            del(league_dict[i[0]][2])
        elif((season_dict[i[0]][3][-6:]==season_dict[i[0]][0][-6:]) or (season_dict[i[0]][3][-6:]==season_dict[i[0]][1][-6:])):
            del(season_dict[i[0]][3])
            del(league_dict[i[0]][3])
        elif((season_dict[i[0]][4][-6:]==season_dict[i[0]][0][-6:]) or (season_dict[i[0]][4][-6:]==season_dict[i[0]][1][-6:])):
            del(season_dict[i[0]][4])
            del(league_dict[i[0]][4])

In [169]:
season_dict['1978'].remove('PSV Eindhoven')
season_dict['1986'].remove('1. FC Köln')

### Convert the list of dictionaries into a self-contained data-frame which has winners, runner-ups and finalists for each season

In [170]:
europa_league_data = pd.DataFrame(list(season_dict.items()))
europa_league_data.columns = ['Season','Teams']
winner_list=[]
runner_list=[]
sf1_list=[]
sf2_list=[]
for i in range(0,len(europa_league_data)):
    winner_list.append(europa_league_data.Teams[i][0])
    runner_list.append(europa_league_data.Teams[i][1])
    sf1_list.append(europa_league_data.Teams[i][2])
    sf2_list.append(europa_league_data.Teams[i][3])
europa_league_data['Winners'] = winner_list
europa_league_data['Runner Up'] = runner_list
europa_league_data['SF1'] = sf1_list
europa_league_data['SF2'] = sf2_list
europa_league_data = europa_league_data.drop(columns='Teams')
europa_league_data = europa_league_data.sort_values('Season')
europa_league_data = europa_league_data.set_index('Season')

In [171]:
europa_league_teams = pd.DataFrame(list(league_dict.items()))
europa_league_teams.columns = ['Season','Teams']
winner_list=[]
runner_list=[]
sf1_list=[]
sf2_list=[]
for i in range(0,len(europa_league_teams)):
    winner_list.append(europa_league_teams.Teams[i][0])
    runner_list.append(europa_league_teams.Teams[i][1])
    sf1_list.append(europa_league_teams.Teams[i][2])
    sf2_list.append(europa_league_teams.Teams[i][3])
europa_league_teams['Winners'] = winner_list
europa_league_teams['Runner Up'] = runner_list
europa_league_teams['SF1'] = sf1_list
europa_league_teams['SF2'] = sf2_list
europa_league_teams = europa_league_teams.drop(columns='Teams')
europa_league_teams = europa_league_teams.sort_values('Season')
europa_league_teams = europa_league_teams.set_index('Season')

In [172]:
unique_leagues = europa_league_teams['Winners'].unique()
unique_leagues = np.append(unique_leagues,europa_league_teams['Runner Up'].unique())
unique_leagues = np.append(unique_leagues,europa_league_teams['SF1'].unique())
unique_leagues = np.append(unique_leagues,europa_league_teams['SF2'].unique())
lg_unique = np.unique(unique_leagues)
header = ['range1win','range2win','range3win','range4win','range5win','range1run','range2run','range3run','range4run','range5run','range1sf1','range2sf1','range3sf1','range4sf1','range5sf1','range1sf2','range2sf2','range3sf2','range4sf2','range5sf2']
league_df = pd.DataFrame(columns=header,index=lg_unique)
league_df = league_df.fillna(0)

In [173]:
range1_start = int(europa_league_teams.index.max())
range2_start = range1_start-3
range3_start = range2_start-3
range4_start = range3_start-3
range5_start = range4_start-3
range_dict = {"range1_start":range1_start,"range2_start":range2_start,"range3_start":range3_start,"range4_start":range4_start,"range5_start":range5_start}

In [174]:
for i in europa_league_teams.index:
    if (range_dict['range1_start'] >= int(i) > range_dict['range2_start']):
        winning_league = (europa_league_teams.loc[i]['Winners'])
        runner_league = (europa_league_teams.loc[i]['Runner Up'])
        sf1_league = (europa_league_teams.loc[i]['SF1'])
        sf2_league = (europa_league_teams.loc[i]['SF2'])
        league_df.loc[winning_league]['range1win'] = (league_df.loc[winning_league]['range1win']) + 1
        league_df.loc[runner_league]['range1run'] = (league_df.loc[runner_league]['range1run']) + 1
        league_df.loc[sf1_league]['range1sf1'] = (league_df.loc[sf1_league]['range1sf1']) + 1
        league_df.loc[sf2_league]['range1sf2'] = (league_df.loc[sf2_league]['range1sf2']) + 1
    elif (range_dict['range2_start'] >= int(i) > range_dict['range3_start']):
        winning_league = (europa_league_teams.loc[i]['Winners'])
        runner_league = (europa_league_teams.loc[i]['Runner Up'])
        sf1_league = (europa_league_teams.loc[i]['SF1'])
        sf2_league = (europa_league_teams.loc[i]['SF2'])
        league_df.loc[winning_league]['range2win'] = (league_df.loc[winning_league]['range2win']) + 1
        league_df.loc[runner_league]['range2run'] = (league_df.loc[runner_league]['range2run']) + 1
        league_df.loc[sf1_league]['range2sf1'] = (league_df.loc[sf1_league]['range2sf1']) + 1
        league_df.loc[sf2_league]['range2sf2'] = (league_df.loc[sf2_league]['range2sf2']) + 1
    elif (range_dict['range3_start'] >= int(i) > range_dict['range4_start']):
        winning_league = (europa_league_teams.loc[i]['Winners'])
        runner_league = (europa_league_teams.loc[i]['Runner Up'])
        sf1_league = (europa_league_teams.loc[i]['SF1'])
        sf2_league = (europa_league_teams.loc[i]['SF2'])
        league_df.loc[winning_league]['range3win'] = (league_df.loc[winning_league]['range3win']) + 1
        league_df.loc[runner_league]['range3run'] = (league_df.loc[runner_league]['range3run']) + 1
        league_df.loc[sf1_league]['range3sf1'] = (league_df.loc[sf1_league]['range3sf1']) + 1
        league_df.loc[sf2_league]['range3sf2'] = (league_df.loc[sf2_league]['range3sf2']) + 1
    elif (range_dict['range4_start'] >= int(i) > range_dict['range5_start']):
        winning_league = (europa_league_teams.loc[i]['Winners'])
        runner_league = (europa_league_teams.loc[i]['Runner Up'])
        sf1_league = (europa_league_teams.loc[i]['SF1'])
        sf2_league = (europa_league_teams.loc[i]['SF2'])
        league_df.loc[winning_league]['range4win'] = (league_df.loc[winning_league]['range4win']) + 1
        league_df.loc[runner_league]['range4run'] = (league_df.loc[runner_league]['range4run']) + 1
        league_df.loc[sf1_league]['range4sf1'] = (league_df.loc[sf1_league]['range4sf1']) + 1
        league_df.loc[sf2_league]['range4sf2'] = (league_df.loc[sf2_league]['range4sf2']) + 1
    elif (range_dict['range5_start'] >= int(i) >= range_dict['range5_start']-2):
        winning_league = (europa_league_teams.loc[i]['Winners'])
        runner_league = (europa_league_teams.loc[i]['Runner Up'])
        sf1_league = (europa_league_teams.loc[i]['SF1'])
        sf2_league = (europa_league_teams.loc[i]['SF2'])
        league_df.loc[winning_league]['range5win'] = (league_df.loc[winning_league]['range5win']) + 1
        league_df.loc[runner_league]['range5run'] = (league_df.loc[runner_league]['range5run']) + 1
        league_df.loc[sf1_league]['range5sf1'] = (league_df.loc[sf1_league]['range5sf1']) + 1
        league_df.loc[sf2_league]['range5sf2'] = (league_df.loc[sf2_league]['range5sf2']) + 1

In [175]:
league_df = league_df.fillna(0)

In [180]:
# league_df.to_csv('..\\Consolidated Data\\Europa League Data Consildation.csv')